# 필요 패키지 로드

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# 데이터 전처리

In [14]:
data = pd.read_csv('data_8074.csv', encoding = 'utf-8')

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8074 entries, 0 to 8073
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8074 non-null   int64  
 1   icls_cnts   8074 non-null   object 
 2   label       8074 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 189.4+ KB


In [16]:
data['label'].replace(999,0,inplace = True)

In [17]:
df = data[['icls_cnts','label']]
df

,icls_cnts,label
0,..<b>경찰</b>과 추격전까지 벌인 40대 남성이 붙잡혔습니다. .. 1시간이 ...,1.0
1,순창<b>경찰</b>서 사회적 약자 발굴·지원 앞장 .. 순창<b>경찰</b>서(서...,1.0
2,그 생김새가 미심쩍었던 부부는 일주일 넘게 고양이를 데리고 있다 <b>경찰</b>...,1.0
3,"수원남부署, 모범<b>경찰</b> 최보라·김세홍 경장에 표창·장려장 수원남부<b>경...",1.0
4,조두순에 대한 1대1 전자감독 및 <b>경찰</b>을 통한 24시간 밀착 감독 등...,1.0
...,...,...
8069,김주원 음성경찰서 설성지구대장(왼쪽)이 전화금융사기를 예방한 음성농협 원남지점 서아...,0.0
8070,..경찰청◎총경 승진 예정△서울 광역수사 광역1 박종환△광주 형사 강력 송기주△전남...,0.0
8071,"대구경찰청, 2021년 설 명절 종합치안활동 추진 대구경찰청이 오는 14일까지 20...",0.0
8072,..경찰청에 따르면 경찰은 지난달 30일 오후 9시 30분쯤 부산 중구 한 상점에서...,0.0


In [18]:
data.drop_duplicates(subset = ['icls_cnts'], inplace = True)
data['label'].value_counts()

 0.0    5320
 1.0    1783
-1.0     971
Name: label, dtype: int64

In [39]:
print('결측값 여부:',df.isnull().values.any())

결측값 여부: False


# 문장 정제 및 토큰화

In [20]:
import re 
#텍스트 정제 함수 : 한글 이외의 문자 제거
def text_cleaning(text):
    #한글 정규표현식으로 한글만 추출
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+')
    result = hangul.sub('',text)
    return result

In [21]:
#만든 함수 적용
df['ko_text']=df['icls_cnts'].apply(lambda x: text_cleaning(x))
df.head

<bound method NDFrame.head of                                               icls_cnts  label  \
0     ..<b>경찰</b>과 추격전까지 벌인 40대 남성이 붙잡혔습니다. .. 1시간이 ...    1.0   
1     순창<b>경찰</b>서 사회적 약자 발굴·지원 앞장 ..　순창<b>경찰</b>서(서...    1.0   
2      그 생김새가 미심쩍었던 부부는 일주일 넘게 고양이를 데리고 있다 <b>경찰</b>...    1.0   
3     수원남부署, 모범<b>경찰</b> 최보라·김세홍 경장에 표창·장려장 수원남부<b>경...    1.0   
4      조두순에 대한 1대1 전자감독 및 <b>경찰</b>을 통한 24시간 밀착 감독 등...    1.0   
...                                                 ...    ...   
8069  김주원 음성경찰서 설성지구대장(왼쪽)이 전화금융사기를 예방한 음성농협 원남지점 서아...    0.0   
8070  ..경찰청◎총경 승진 예정△서울 광역수사 광역1 박종환△광주 형사 강력 송기주△전남...    0.0   
8071  대구경찰청, 2021년 설 명절 종합치안활동 추진 대구경찰청이 오는 14일까지 20...    0.0   
8072  ..경찰청에 따르면 경찰은 지난달 30일 오후 9시 30분쯤 부산 중구 한 상점에서...    0.0   
8073  "사각지대"에 놓인 불법 유흥시설에 대해 경찰이 대대적인 단속에 나선 가운데 1주 ...    0.0   

                                                ko_text  
0     경찰과추격전까지벌인대남성이붙잡혔습니다시간이넘도록음주운전차량을뒤쫓던경찰은결국실탄을쏴시...  
1     순창경찰서사회적약자발굴지원앞장순창경찰서서장정재봉가지난해부터시행하는경찰에따르면이사업은...  
2  

In [22]:
from konlpy.tag import Okt

In [23]:
#okt 라이브러리로 텍스트 데이터에서 형태소 추출
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word,tag) for word, tag in pos]
    return pos

#함수 테스트 
result = get_pos(df['ko_text'][0])
print(result)

['경찰/Noun', '과/Josa', '추/Noun', '격전/Noun', '까지/Josa', '벌인대/Verb', '남성/Noun', '이/Josa', '붙잡혔습니다/Verb', '시간/Noun', '이/Josa', '넘도록/Verb', '음주운전/Noun', '차량/Noun', '을/Josa', '뒤/Noun', '쫓던/Verb', '경찰/Noun', '은/Josa', '결국/Adverb', '실탄/Noun', '을/Josa', '쏴/Verb', '시간/Noun', '이/Josa', '넘도록/Verb', '음주운전/Noun', '차량/Noun', '을/Josa', '뒤/Noun', '쫓던/Verb', '경찰/Noun', '은/Josa', '결국/Adverb', '실탄/Noun', '을/Josa', '쏴/Verb', '차/Noun', '를/Josa', '세웠는데/Verb', '잡고/Noun', '보니이/Verb', '남성/Noun', '운전면허/Noun', '가/Josa', '없었습니다/Adjective', '경찰서/Noun', '경찰/Noun', '은/Josa', '도로/Noun', '교통법위반/Noun', '등/Noun', '의/Josa', '혐의/Noun', '로/Josa', '살/Noun', '씨/Suffix', '에/Josa', '대해/Noun', '구속영장/Noun', '을/Josa', '신청/Noun', '할/Verb', '계획/Noun', '입니다/Adjective']


# 정수화(Countvectorizer활용), TF-IDF로 중요 단어 추출

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
#형태소를 벡터 형태의 학습 데이터셋으로 변환
index_vectorizer = CountVectorizer(tokenizer = lambda x : get_pos(x))
X = index_vectorizer.fit_transform(df['ko_text'].tolist())

In [26]:
X.shape

(8074, 35370)

In [27]:
print(str(index_vectorizer.vocabulary_)[:100]+'..')

{'경찰/Noun': 1765, '과/Josa': 2367, '추/Noun': 30413, '격전/Noun': 1530, '까지/Josa': 4219, '벌인대/Verb': 137..


In [28]:
print(df['ko_text'][0])
print(X[0])

경찰과추격전까지벌인대남성이붙잡혔습니다시간이넘도록음주운전차량을뒤쫓던경찰은결국실탄을쏴시간이넘도록음주운전차량을뒤쫓던경찰은결국실탄을쏴차를세웠는데잡고보니이남성운전면허가없었습니다경찰서경찰은도로교통법위반등의혐의로살씨에대해구속영장을신청할계획입니다
  (0, 1765)	4
  (0, 2367)	1
  (0, 30413)	1
  (0, 1530)	1
  (0, 4219)	1
  (0, 13707)	1
  (0, 5052)	2
  (0, 24037)	3
  (0, 15266)	1
  (0, 18629)	2
  (0, 5553)	2
  (0, 23870)	2
  (0, 29377)	2
  (0, 23820)	5
  (0, 8690)	2
  (0, 29269)	2
  (0, 23735)	3
  (0, 1581)	2
  (0, 19163)	2
  (0, 19445)	2
  (0, 29347)	1
  (0, 9976)	1
  (0, 17571)	1
  (0, 26343)	1
  (0, 14083)	1
  (0, 22962)	1
  (0, 4)	1
  (0, 21242)	1
  (0, 1779)	1
  (0, 7555)	1
  (0, 2708)	1
  (0, 9138)	1
  (0, 23896)	1
  (0, 34439)	1
  (0, 9836)	1
  (0, 16324)	1
  (0, 19549)	1
  (0, 21292)	1
  (0, 7324)	1
  (0, 2816)	1
  (0, 19070)	1
  (0, 33307)	1
  (0, 1856)	1
  (0, 25685)	1


In [29]:
#중요 단어 추출을 위해 TF-IDF 활용
from sklearn.feature_extraction.text import TfidfTransformer

In [30]:
tfidf_vectorizer = TfidfTransformer()
X = tfidf_vectorizer.fit_transform(X)

In [31]:
print(X.shape)
print(X[0])

(8074, 35370)
  (0, 34439)	0.056720501413239165
  (0, 33307)	0.06759964537134064
  (0, 30413)	0.12642961523183818
  (0, 29377)	0.1666966856318872
  (0, 29347)	0.09697326658404562
  (0, 29269)	0.3840522635730061
  (0, 26343)	0.14073649703171057
  (0, 25685)	0.10008632152629426
  (0, 24037)	0.09388562836587759
  (0, 23896)	0.03160157510643609
  (0, 23870)	0.2013904580574777
  (0, 23820)	0.11622090759890523
  (0, 23735)	0.09267680887858244
  (0, 22962)	0.14349264739115963
  (0, 21292)	0.025727903548247014
  (0, 21242)	0.17771924998663066
  (0, 19549)	0.06784983591853408
  (0, 19445)	0.2911168335142245
  (0, 19163)	0.2503565894003772
  (0, 19070)	0.09463410954684548
  (0, 18629)	0.1713195690154195
  (0, 17571)	0.19202613178650305
  (0, 16324)	0.10894099134671784
  (0, 15266)	0.14253245252391808
  (0, 14083)	0.19202613178650305
  (0, 13707)	0.19202613178650305
  (0, 9976)	0.027124867560636406
  (0, 9836)	0.04083843304212885
  (0, 9138)	0.041734928201384976
  (0, 8690)	0.1578924569187968
  (

# 모델 적용

In [36]:
from sklearn.model_selection import train_test_split

y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1)
print(X_train.shape)
print(X_test.shape)

(7266, 35370)
(808, 35370)


In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_pred_probability = lr.predict_proba(X_test)[:,1]

print("정확도:%.2f"% accuracy_score(y_test, y_pred))
print("정밀도:%.3f"% precision_score(y_test,y_pred, average = 'weighted'))
print("재현율:%.3f"% recall_score(y_test, y_pred, average = 'weighted'))
print("F1:%.3f"% f1_score(y_test,y_pred, average = 'weighted'))

정확도:0.77
정밀도:0.765
재현율:0.767
F1:0.752


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
